In [1]:
# Copyright 2019 Luke Pinner
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     http://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from contextlib import contextmanager

import rasterio
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling


@contextmanager
def resample_raster(raster, out_path=None, scale=2):
    """ Resample a raster
        multiply the pixel size by the scale factor
        divide the dimensions by the scale factor
        i.e
        given a pixel size of 250m, dimensions of (1024, 1024) and a scale of 2,
        the resampled raster would have an output pixel size of 500m and dimensions of (512, 512)
        given a pixel size of 250m, dimensions of (1024, 1024) and a scale of 0.5,
        the resampled raster would have an output pixel size of 125m and dimensions of (2048, 2048)
        returns a DatasetReader instance from either a filesystem raster or MemoryFile (if out_path is None)
    """
    t = raster.transform

    # rescale the metadata
    transform = Affine(t.a * scale, t.b, t.c, t.d, t.e * scale, t.f)
    height = int(raster.height / scale)
    width = int(raster.width / scale)

    profile = src.profile
    profile.update(transform=transform, driver='GTiff', height=height, width=width)

    data = raster.read(
            out_shape=(raster.count, height, width),
            resampling=Resampling.bilinear,
        )

    if out_path is None:
        with write_mem_raster(data, **profile) as dataset:
            del data
            yield dataset

    else:
        with write_raster(out_path, data, **profile) as dataset:
            del data
            yield dataset


@contextmanager
def write_mem_raster(data, **profile):
    with MemoryFile() as memfile:
        with memfile.open(**profile) as dataset:  # Open as DatasetWriter
            dataset.write(data)

        with memfile.open() as dataset:  # Reopen as DatasetReader
            yield dataset  # Note yield not return


@contextmanager
def write_raster(path, data, **profile):

    with rasterio.open(path, 'w', **profile) as dataset:  # Open as DatasetWriter
        dataset.write(data)

    with rasterio.open(path) as dataset:  # Reopen as DatasetReader
        yield dataset

In [2]:
raster = rasterio.open("/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/data.2008.081.tiff")

In [4]:
resample_raster(raster, out_path=None, scale=2)

In [3]:
import gdal

ds = gdal.Warp('warp_test.tif', "/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/data.2008.081.tiff", dstSRS='EPSG:4326',
               outputType=gdal.GDT_Int16, xRes=0.00892857142857143, yRes=0.00892857142857143)

In [5]:
ds = None

In [6]:
ds

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xesmf as xe

In [5]:
ds = xr.tutorial.open_dataset('air_temperature') # use xr.tutorial.load_dataset() for xarray<v0.11.0
ds

<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53, time: 2920)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [9]:
dr = ds['air'] 
dr

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [ ]:
regridder = xe.Regridder(ds_in, ds_out, 'bilinear')

In [6]:
data = xr.open_rasterio("/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/data.2008.081.tiff")

In [7]:
data

<xarray.DataArray (band: 1, y: 32739, x: 29838)>
[976866282 values with dtype=uint8]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 40.0 40.0 39.99 39.99 ... -38.99 -38.99 -39.0 -39.0
  * x        (x) float64 -20.0 -20.0 -19.99 -19.99 ... 51.99 51.99 52.0 52.0
Attributes:
    transform:      (0.0024129999999999998, 0.0, -19.999999999999975, 0.0, -0...
    crs:            +init=epsg:4326
    res:            (0.0024129999999999998, 0.002413)
    is_tiled:       1
    nodatavals:     (0.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    DataType:       Thematic

In [12]:
data['y'].values, data['x'].values

(array([ 39.99879415,  39.99638115,  39.99396815, ..., -38.99317385,
        -38.99558685, -38.99799985]),
 array([-19.9987935, -19.9963805, -19.9939675, ...,  51.9930615,
         51.9954745,  51.9978875]))

In [18]:
data_out = xr.Dataset({'lat': (['lat'], np.arange(-39, 40, 0.1)),
                     'lon': (['lon'], np.arange(-20, 52, 0.1)),
                    }
                   )
data_out

<xarray.Dataset>
Dimensions:  (lat: 790, lon: 720)
Coordinates:
  * lat      (lat) float64 -39.0 -38.9 -38.8 -38.7 -38.6 ... 39.6 39.7 39.8 39.9
  * lon      (lon) float64 -20.0 -19.9 -19.8 -19.7 -19.6 ... 51.6 51.7 51.8 51.9
Data variables:
    *empty*

In [19]:
regridder = xe.Regridder(data, data_out, 'bilinear')

KeyError: 'lon'